# Tutorial 06, case 5: Stokes problem with distributed control

In this tutorial we solve the optimal control problem

$$\min J(y, u) = \frac{1}{2} \int_{\Omega} |v - v_d|^2 dx + \frac{\alpha}{2} \int_{\Omega} |u|^2 dx$$
s.t.
$$\begin{cases}
- \Delta v + \nabla p = f + u   & \text{in } \Omega\\
         \text{div} v = 0       & \text{in } \Omega\\
                    v = 0       & \text{on } \partial\Omega
\end{cases}$$

where
$$\begin{align*}
& \Omega                      & \text{unit square}\\
& u \in [L^2(\Omega)]^2       & \text{control variable}\\
& v \in [H^1_0(\Omega)]^2     & \text{state velocity variable}\\
& p \in L^2(\Omega)           & \text{state pressure variable}\\
& \alpha > 0                  & \text{penalization parameter}\\
& v_d                         & \text{desired state}\\
& f                           & \text{forcing term}
\end{align*}$$
using an adjoint formulation solved by a one shot approach

In [ ]:
import numpy as np
from mpi4py import MPI
from petsc4py import PETSc
import sympy
from ufl import div, grad, inner, Measure, replace, TestFunction, TrialFunction
from dolfinx import Constant, DirichletBC, Function, FunctionSpace, VectorFunctionSpace
from dolfinx.fem import locate_dofs_topological
from dolfinx.io import XDMFFile
from dolfinx.plot import create_vtk_topology
from multiphenicsx.fem import (assemble_matrix_block, assemble_scalar, assemble_vector_block,
                               BlockVecSubVectorWrapper, create_vector_block)
import pyvista

### Mesh

In [ ]:
with XDMFFile(MPI.COMM_WORLD, "data/square.xdmf", "r") as infile:
    mesh = infile.read_mesh()
    mesh.topology.create_connectivity_all()
    subdomains = infile.read_meshtags(mesh, name="subdomains")
    boundaries = infile.read_meshtags(mesh, name="boundaries")
boundaries_1234 = boundaries.indices[np.isin(boundaries.values, (1, 2, 3, 4))]

In [ ]:
# Define associated measures
dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)

In [ ]:
def dolfinx_to_pyvista_mesh(mesh):
    num_cells = mesh.topology.index_map(mesh.topology.dim).size_local
    cell_entities = np.arange(num_cells, dtype=np.int32)
    pyvista_cells, cell_types = create_vtk_topology(mesh, mesh.topology.dim, cell_entities)
    grid = pyvista.UnstructuredGrid(pyvista_cells, cell_types, mesh.geometry.x)
    return grid

In [ ]:
def pyvista_mesh_plot(mesh):
    grid = dolfinx_to_pyvista_mesh(mesh)
    plotter = pyvista.PlotterITK()
    plotter.add_mesh(grid)
    plotter.show()

In [ ]:
pyvista_mesh_plot(mesh)

### Function spaces

In [ ]:
Y_velocity = VectorFunctionSpace(mesh, ("Lagrange", 2))
Y_pressure = FunctionSpace(mesh, ("Lagrange", 1))
U = VectorFunctionSpace(mesh, ("Lagrange", 2))
Q_velocity = Y_velocity.clone()
Q_pressure = Y_pressure.clone()

### Trial and test functions

In [ ]:
(v, p) = (TrialFunction(Y_velocity), TrialFunction(Y_pressure))
(w, q) = (TestFunction(Y_velocity), TestFunction(Y_pressure))
u = TrialFunction(U)
r = TestFunction(U)
(z, b) = (TrialFunction(Q_velocity), TrialFunction(Q_pressure))
(s, d) = (TestFunction(Q_velocity), TestFunction(Q_pressure))

 ### Problem data

In [ ]:
alpha = 1.e-5
epsilon = 1.e-5
x, y = sympy.symbols("x[0], x[1]")
psi_d = 10 * (1 - sympy.cos(0.8 * np.pi * x)) * (1 - sympy.cos(0.8 * np.pi * y)) * (1 - x)**2 * (1 - y)**2
v_d_x = sympy.lambdify([x, y], psi_d.diff(y, 1))
v_d_y = sympy.lambdify([x, y], - psi_d.diff(x, 1))
v_d = Function(Y_velocity)
v_d.interpolate(lambda x: np.stack((v_d_x(x[0], x[1]), v_d_y(x[0], x[1])), axis=0))
ff = Constant(mesh, (0., 0.))
bc0 = Function(Y_velocity)

### Optimality conditions

In [ ]:
a = [[inner(v, w) * dx, None, None, inner(grad(z), grad(w)) * dx, - b * div(w) * dx],
     [None, None, None, - q * div(z) * dx, epsilon * b * q * dx],
     [None, None, alpha * inner(u, r) * dx, - inner(z, r) * dx, None],
     [inner(grad(v), grad(s)) * dx, - p * div(s) * dx, - inner(u, s) * dx, None, None],
     [- d * div(v) * dx, epsilon * p * d * dx, None, None, None]]
f = [inner(v_d, w) * dx,
     None,
     None,
     inner(ff, s) * dx,
     None]
a[3][3] = Constant(mesh, 0.) * inner(z, s) * dx
f[1] = Constant(mesh, 0.) * q * dx
f[2] = inner(Constant(mesh, (0., 0.)), r) * dx
f[4] = Constant(mesh, 0.) * d * dx
bdofs_Y_velocity_1234 = locate_dofs_topological((Y_velocity, Y_velocity), mesh.topology.dim - 1, boundaries_1234)
bdofs_Q_velocity_1234 = locate_dofs_topological((Q_velocity, Y_velocity), mesh.topology.dim - 1, boundaries_1234)
bc = [DirichletBC(bc0, bdofs_Y_velocity_1234, Y_velocity),
      DirichletBC(bc0, bdofs_Q_velocity_1234, Q_velocity)]

### Solution

In [ ]:
(v, p) = (Function(Y_velocity), Function(Y_pressure))
u = Function(U)
(z, b) = (Function(Q_velocity), Function(Q_pressure))

### Cost functional

In [ ]:
J = 0.5 * inner(v - v_d, v - v_d) * dx + 0.5 * alpha * inner(u, u) * dx

### Uncontrolled functional value

In [ ]:
# Extract state forms from the optimality conditions
a_state = [[replace(a[i][j], {s: w, d: q}) if a[i][j] is not None else None
            for j in (0, 1)] for i in (3, 4)]
f_state = [replace(f[i], {s: w, d: q}) for i in (3, 4)]
bc_state = [bc[0]]

In [ ]:
# Assemble the block linear system for the state
A_state = assemble_matrix_block(a_state, bcs=bc_state)
A_state.assemble()
F_state = assemble_vector_block(f_state, a_state, bcs=bc_state)

In [ ]:
# Solve
vp = create_vector_block([f[j] for j in (0, 1)])
ksp = PETSc.KSP()
ksp.create(mesh.mpi_comm())
ksp.setOperators(A_state)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.setFromOptions()
ksp.solve(F_state, vp)
vp.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
# Split the block solution in components
with BlockVecSubVectorWrapper(vp, [c.function_space.dofmap for c in (v, p)]) as vp_wrapper:
    for vp_wrapper_local, component in zip(vp_wrapper, (v, p)):
        with component.vector.localForm() as component_local:
            component_local[:] = vp_wrapper_local

In [ ]:
J_uncontrolled = mesh.mpi_comm().allreduce(assemble_scalar(J), op=MPI.SUM)
print("Uncontrolled J =", J_uncontrolled)
assert np.isclose(J_uncontrolled, 0.1784536)

In [ ]:
def pyvista_scalar_field_plot(mesh, scalar_field, name):
    grid = dolfinx_to_pyvista_mesh(mesh)
    grid.point_arrays[name] = scalar_field.compute_point_values()
    grid.set_active_scalars(name)
    plotter = pyvista.PlotterITK()
    plotter.add_mesh(grid)
    plotter.show()

In [ ]:
def pyvista_vector_field_plot(mesh, vector_field, name, factor):
    grid = dolfinx_to_pyvista_mesh(mesh)
    values = np.zeros((mesh.geometry.x.shape[0], 3))
    values[:, :2] = vector_field.compute_point_values()
    grid.point_arrays[name] = values
    grid.set_active_vectors(name)
    plotter = pyvista.PlotterITK()
    plotter.add_mesh(grid)
    glyphs = grid.glyph(orient=name, factor=factor)
    plotter.add_mesh(glyphs)
    plotter.show()

In [ ]:
pyvista_vector_field_plot(mesh, v, "uncontrolled state velocity", factor=1e-1)

In [ ]:
pyvista_scalar_field_plot(mesh, p, "uncontrolled state pressure")

### Optimal control

In [ ]:
# Assemble the block linear system for the optimality conditions
A = assemble_matrix_block(a, bcs=bc)
A.assemble()
F = assemble_vector_block(f, a, bcs=bc)

In [ ]:
# Solve
vpuzb = create_vector_block(f)
ksp = PETSc.KSP()
ksp.create(mesh.mpi_comm())
ksp.setOperators(A)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.setFromOptions()
ksp.solve(F, vpuzb)
vpuzb.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
# Split the block solution in components
with BlockVecSubVectorWrapper(vpuzb, [c.function_space.dofmap for c in (v, p, u, z, b)]) as vpuzb_wrapper:
    for vpuzb_wrapper_local, component in zip(vpuzb_wrapper, (v, p, u, z, b)):
        with component.vector.localForm() as component_local:
            component_local[:] = vpuzb_wrapper_local

In [ ]:
J_controlled = mesh.mpi_comm().allreduce(assemble_scalar(J), op=MPI.SUM)
print("Optimal J =", J_controlled)
assert np.isclose(J_controlled, 0.0052941)

In [ ]:
pyvista_vector_field_plot(mesh, v, "state velocity", factor=1e-1)

In [ ]:
pyvista_scalar_field_plot(mesh, p, "state pressure")

In [ ]:
pyvista_vector_field_plot(mesh, u, "control", factor=1e-3)

In [ ]:
pyvista_vector_field_plot(mesh, z, "adjoint velocity", factor=1e2)

In [ ]:
pyvista_scalar_field_plot(mesh, b, "adjoint pressure")